#### Get the frequencies ranges and get the automated report
1. Get the frequencies and divide them (done)
2. Now traverse the whole dataset and get the needed indices (done)
3. Now for a particular frequency range get the needed values based on some computations needed
4. Finally store the values in the .csv files


In [1]:
# Import all the tools we need
import numpy as np
import pandas as pd
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM, LinearSVC, SVC, SVR
from sklearn.metrics import accuracy_score

In [2]:
analysis = pd.read_csv('CompleteReportOnClutter.csv')
analysis.describe()

,Target,TOFA,TOLA,Hit Count,Frequency(MHz),Pulse Width(us),MaxAmplitude(dBm),AOA(Deg),DOA(Deg),DF Type,...,Yaw(deg),IsPassive,BI Phase Flag,Est_LF_Lat(deg),Est_LF_Long(deg),Est_Major_Axis(kms),Est_Minor_Axis(kms),Inclinationarr_fPRI[0],arr_fPRI[1],TD
count,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,...,11840.000000,11840.0,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000,11840.000000
mean,0.432264,52673.908446,52987.662753,2085.345355,5209.313803,8.954191,-62.061202,181.494018,183.874688,1.939780,...,-116.848218,0.0,0.181757,5.175648,34.270829,40.419043,2.878245,74.064539,1512.293047,313.754307
std,0.495411,192.100954,41.747266,1671.892403,3221.715706,20.497141,10.460356,105.782496,91.529191,0.997254,...,2.558922,0.0,0.385660,5.955387,38.099100,51.366448,6.028200,104.009052,1074.718683,198.716139
min,0.000000,52411.000000,52904.000000,51.000000,1250.952148,0.181667,-74.907372,36.345985,31.568359,1.000000,...,-118.608391,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.600000,1.000000
25%,0.000000,52506.000000,52950.000000,438.500000,2799.853516,0.655000,-70.783554,58.069550,112.127747,1.000000,...,-117.916260,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,889.084000,136.000000
50%,0.000000,52620.000000,52977.000000,2081.000000,3457.833426,0.996818,-64.753014,175.636429,190.619385,2.000000,...,-117.586662,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1340.160000,330.000000
75%,1.000000,52811.000000,53022.000000,2775.750000,9245.894401,4.295000,-53.818363,288.953217,287.213074,3.000000,...,-117.515259,0.0,0.000000,11.322827,75.508377,60.183659,2.102043,191.623001,2222.100000,496.000000
max,1.000000,52962.000000,53072.000000,6194.000000,9422.078933,184.314987,-32.047184,336.263000,305.566284,5.000000,...,-106.446533,0.0,1.000000,14.371135,79.498367,164.308411,29.679119,281.945343,4364.800000,647.000000


1. Target	
2. Duration Of Intercept	
3. Max TOLA	Max Hit Count	
4. Min Frequency(MHz)	
5. Max Frequency(MHz)	
6. Max Pulse Width(us)	
7. Max arr_fPRI[1]	
8. Max Bandwidth(MHz)	
9. Est_LF_Lat(deg)	
10. Est_LF_Long(deg)	
11. Min Amplitude(dBm)	
12. Max Amplitude(dBm)


In [5]:
analysis_1 = analysis[analysis['Target'] == 1]

In [6]:
analysis_arr = np.array(analysis_1)

In [7]:
analysis_arr

array([[1.00000000e+00, 5.24110000e+04, 5.29220000e+04, ...,
        2.69989620e+01, 4.26602000e+02, 5.11000000e+02],
       [1.00000000e+00, 5.25060000e+04, 5.29040000e+04, ...,
        2.81945343e+02, 7.52525000e+02, 3.98000000e+02],
       [1.00000000e+00, 5.26200000e+04, 5.29200000e+04, ...,
        0.00000000e+00, 1.66661100e+03, 3.00000000e+02],
       ...,
       [1.00000000e+00, 5.26200000e+04, 5.30410000e+04, ...,
        1.13974731e+02, 1.66660400e+03, 4.21000000e+02],
       [1.00000000e+00, 5.24110000e+04, 5.30580000e+04, ...,
        2.69989620e+01, 3.51904000e+02, 6.47000000e+02],
       [1.00000000e+00, 5.26200000e+04, 5.30410000e+04, ...,
        1.13974731e+02, 1.66660400e+03, 4.21000000e+02]])

In [108]:
bin_1 = []
bin_2 = []
bin_3 = []
bin_4 = []
bin_5 = []
bin_6 = []
bin_7 = []
bin_8 = []
for i in range(len(analysis_arr)):
    if 1250 < analysis_arr[i][4] < 1260:
        bin_1.append(i)
    elif 1279 < analysis_arr[i][4] < 1281:
        bin_2.append(i)
    elif 2798 < analysis_arr[i][4] < 2800:
        bin_3.append(i)
    elif 2831 < analysis_arr[i][4] < 2833:
        bin_4.append(i)
    elif 3448 < analysis_arr[i][4] < 4000:
        bin_5.append(i)
    elif 8361 < analysis_arr[i][4] < 8362:
        bin_6.append(i)
    elif 9376 < analysis_arr[i][4] < 9378:
        bin_7.append(i)
    elif 9405 < analysis_arr[i][4] < 9407:
        bin_8.append(i)    

In [109]:
print('Size of bin 1:', len(bin_1))
print('Size of bin 2:', len(bin_2))
print('Size of bin 3:', len(bin_3))
print('Size of bin 4:', len(bin_4))
print('Size of bin 5:', len(bin_5))
print('Size of bin 6:', len(bin_6))
print('Size of bin 7:', len(bin_7))
print('Size of bin 8:', len(bin_8))

Size of bin 1: 373
Size of bin 2: 1357
Size of bin 3: 1273
Size of bin 4: 380
Size of bin 5: 971
Size of bin 6: 572
Size of bin 7: 94
Size of bin 8: 98


In [110]:
max_freq = analysis_arr[bin_1[0]][4]
print("Freq :",max_freq)
for i in range(len(bin_1)):
    max_freq = max(analysis_arr[bin_1[i]][4], max_freq)
print("Freq :",max_freq)

Freq : 1251.196289
Freq : 1251.258263


In [99]:
# Duration Of Intercept	
# Max TOLA
# Max Hit Count	
# Min Frequency(MHz)	
# Max Frequency(MHz)	
# Max Pulse Width(us)	
# Max arr_fPRI[1]	
# Max Bandwidth(MHz)	
# Est_LF_Lat(deg)	
# Est_LF_Long(deg)	
# Min Amplitude(dBm)	
# Max Amplitude(dBm)
for j in range(0, len(analysis_arr)):
    min_tola = 99999
    doi = 0
    max_tola = 0
    max_hit_count = 0 
    max_freq = 0
    min_freq = 9999
    max_pw = []
    max_pri = []
    max_bw = 0
    est_lat = 0
    est_long = 0
    max_amp = -100
    min_amp = 0
    for i in range(0, len(bin_1)):
        max_tola = max(max_tola, analysis_arr[bin_1[i]][2])
        min_tola = min(analysis_arr[bin_1[0]][2], analysis_arr[bin_1[i]][2])
        doi = max_tola - min_tola
        max_hit_count = max(max_hit_count, analysis_arr[bin_1[i]][3])
        max_freq = max(max_freq, analysis_arr[bin_1[i]][4])
        min_freq = min(min_freq, analysis_arr[bin_1[i]][4])
        # max_pw = max(max_pw, analysis_arr[bin_1[i]][5])
        max_pw = set.add(analysis_arr[bin_1[i]][5])
        # max_pri = max(max_pri, analysis_arr[bin_1[i]][29])
        max_pri = set.add(analysis_arr[bin_1[i]][29])
        max_bw = max(max_bw, analysis_arr[bin_1[i]][11])
        est_lat = max(est_lat, analysis_arr[bin_1[i]][24])
        est_long = max(est_long, analysis_arr[bin_1[i]][25])
        max_amp = max(max_amp, analysis_arr[bin_1[i]][6])
        min_amp = min(min_amp, analysis_arr[bin_1[i]][6])
        print(max_pw)
    s1 = pd.Series([doi, max_hit_count, max_freq, min_freq, max_pw, max_pri, max_bw, est_lat, est_long, max_amp, min_amp])
    df = pd.DataFrame([list(s1)], columns= ["doi", "max_hit_count", "Max Freq", "Min Freq", "Max Pw", "Max pri", "Max Bw", "Est Lat", "Est long", "max amp", "min amp"])
    df.to_csv('temp.csv', index=False)

TypeError: descriptor 'add' for 'set' objects doesn't apply to a 'numpy.float64' object

In [ ]:
print("MAX_TOLA:", max_tola)
print("MIN_TOLA", min_tola)
print("DOI:", doi)
print("MAX_HIT_COUNT:", max_hit_count)
print("MAX_FREQ:", max_freq)
print("MIN_FREQ:", min_freq)
print("MAX_PW:", max_pw)
print("MAX_PRI:", max_pri)
print("MAX_BW:", max_bw)
print("EST_LAT:", est_lat)
print("EST_LON:", est_long)
print("MAX_AMP:", max_amp)
print("MAX_AMP:", min_amp)

In [127]:
# Duration Of Intercept	
# Max TOLA
# Max Hit Count	
# Min Frequency(MHz)	
# Max Frequency(MHz)	
# Max Pulse Width(us)	
# Max arr_fPRI[1]	
# Max Bandwidth(MHz)	
# Est_LF_Lat(deg)	
# Est_LF_Long(deg)	
# Min Amplitude(dBm)	
# Max Amplitude(dBm)
setter_pw= set()
setter_pri = set()
for i in range(len(bin_8)):
    setter_pw.add(analysis_arr[bin_8[i]][5])
    setter_pri.add(analysis_arr[bin_8[i]][29])
setter_pw= sorted(setter_pw)
setter_pw

[0.425909]

In [128]:
setter_pri = sorted(setter_pri)
setter_pri

[714.116]

In [131]:
analysis_0 = pd.read_csv('CompleteReportOnClutter.csv')
analysis_0 = analysis_0[analysis_0['Target'] == 0]
analysis_array = np.array(analysis_0)
analysis_array

array([[0.00000000e+00, 5.27010000e+04, 5.29290000e+04, ...,
        1.99613892e+02, 9.09431000e+02, 2.28000000e+02],
       [0.00000000e+00, 5.28870000e+04, 5.29090000e+04, ...,
        0.00000000e+00, 4.30660000e+03, 2.20000000e+01],
       [0.00000000e+00, 5.29080000e+04, 5.29110000e+04, ...,
        0.00000000e+00, 1.44812000e+03, 3.00000000e+00],
       ...,
       [0.00000000e+00, 5.28110000e+04, 5.30510000e+04, ...,
        1.91623001e+02, 2.22208800e+03, 2.40000000e+02],
       [0.00000000e+00, 5.29080000e+04, 5.30480000e+04, ...,
        0.00000000e+00, 1.44812000e+03, 1.40000000e+02],
       [0.00000000e+00, 5.29460000e+04, 5.30590000e+04, ...,
        0.00000000e+00, 1.77628000e+03, 1.13000000e+02]])

In [135]:
bin_1 = []
bin_2 = []
bin_3 = []
bin_4 = []
bin_5 = []
bin_6 = []
bin_7 = []
bin_8 = []
for i in range(len(analysis_array)):
    if 2831 < analysis_array[i][4] < 2833:
        bin_1.append(i)
    elif 3439 < analysis_array[i][4] < 3458:
        bin_2.append(i)
    elif 3467 < analysis_array[i][4] < 3485:
        bin_3.append(i)
    elif 8361 < analysis_array[i][4] < 8362:
        bin_4.append(i)
    elif 9238 < analysis_array[i][4] < 9249:
        bin_5.append(i)
    elif 9377 < analysis_array[i][4] < 9378:
        bin_6.append(i)
    elif 9387 < analysis_array[i][4] < 9423:
        bin_7.append(i)
    elif 9425 < analysis_array[i][4] < 11000:
        bin_8.append(i)    

In [136]:
print('Size of bin 1:', len(bin_1))
print('Size of bin 2:', len(bin_2))
print('Size of bin 3:', len(bin_3))
print('Size of bin 4:', len(bin_4))
print('Size of bin 5:', len(bin_5))
print('Size of bin 6:', len(bin_6))
print('Size of bin 7:', len(bin_7))
print('Size of bin 8:', len(bin_8))

Size of bin 1: 955
Size of bin 2: 687
Size of bin 3: 1247
Size of bin 4: 398
Size of bin 5: 854
Size of bin 6: 81
Size of bin 7: 2441
Size of bin 8: 0


In [162]:
# Duration Of Intercept	
# Max TOLA
# Max Hit Count	
# Min Frequency(MHz)	
# Max Frequency(MHz)	
# Max Pulse Width(us)	
# Max arr_fPRI[1]	
# Max Bandwidth(MHz)	
# Est_LF_Lat(deg)	
# Est_LF_Long(deg)	
# Min Amplitude(dBm)	
# Max Amplitude(dBm)
setter_pw= set()
setter_pri = set()
for i in range(len(bin_7)):
    setter_pw.add(analysis_array[bin_7[i]][5])
    setter_pri.add(analysis_array[bin_7[i]][29])
setter_pw= sorted(setter_pw)
setter_pw

[0.181667,
 0.198182,
 0.205909,
 0.206111,
 0.206154,
 0.364,
 0.391667,
 0.399167,
 0.423,
 0.42625,
 0.428333,
 0.433462,
 0.449286,
 0.455,
 0.463,
 0.543889,
 0.548333,
 0.566111,
 0.567,
 0.571667,
 0.575,
 0.583889,
 0.811667]

In [189]:
setter_pri = sorted(setter_pri)
setter_pri

[444.538,
 714.12,
 716.36,
 721.913,
 889.084,
 889.087,
 889.093,
 909.431,
 909.433,
 909.436,
 909.44,
 1019.931,
 1020.406,
 1037.728,
 1075.864,
 1077.327,
 1160.387,
 1160.391,
 1394.667]

In [34]:
import matplotlib.pyplot as plt
Graph = setter_pri
xs = [y for y in range(len(Graph))]
plt.plot(xs, Graph)
plt.show()

NameError: name 'setter_pri' is not defined